In [1]:
import pandas as pd
import numpy as np

from datetime import datetime 

import os

import re

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# Ignorar warings
import warnings
warnings.filterwarnings("ignore")

import dotenv
from dotenv import load_dotenv
load_dotenv()

import psycopg2
from psycopg2 import sql


import sys 
sys.path.append(os.path.abspath("../src"))   


import funciones as f

In [2]:
key= os.getenv("key")

In [3]:
conexion=f.iniciar_conexion("localhost","postgres",key,5432,"Streaming_proyect")

In [4]:
cursor=conexion.cursor()
query_df=""" 
SELECT u.id AS id_usuario, user_type AS tipo_usuario, u.recommendation_system AS recomendacion_usuario,c.id AS id_contenido,c.duration AS duracion_contenido,c.content_type AS tipo_contenido,c.rating AS valoracion_contenido,v.id AS id_views, v.user_id AS id_usuario_views, v.content_id AS id_contenido_views, duration_viewed AS minutos_vistos , v.recommendation_system AS recomendacion_views,ui.id AS id_interaccion,ui.user_id AS id_usuario_interaccion,ui.content_id AS id_contenido_interaccion,ui.interaction_type AS tipo_interaccion 
FROM users u 
INNER JOIN "views" v ON u.id = v.user_id 
INNER JOIN user_interactions ui ON u.id = ui.user_id 
INNER JOIN "content" c ON c.id = v.content_id;

"""
lista_encabezados=['id_usuario', 'tipo_usuario', 'recomendacion_usuario', 'id_contenido', 'duracion_contenido', 'tipo_contenido', 
 'valoracion_contenido', 'numero_views', 'id_usuario_views', 'id_contenido_views', 'minutos_vistos', 
 'recomendacion_views', 'numero_interaccion', 'id_usuario_interaccion', 'id_contenido_interaccion', 'tipo_interaccion']


cursor.execute(query_df)   
df=pd.DataFrame(cursor.fetchall(), columns= lista_encabezados)


Me he dado cuenta de que los minutos visualizados a veces son mayores que la duración del propio contenido

In [5]:
df[df["id_usuario"]== 2]

,id_usuario,tipo_usuario,recomendacion_usuario,id_contenido,duracion_contenido,tipo_contenido,valoracion_contenido,numero_views,id_usuario_views,id_contenido_views,minutos_vistos,recomendacion_views,numero_interaccion,id_usuario_interaccion,id_contenido_interaccion,tipo_interaccion
1887,2,moderator,Recomendeitor,102,23,documentary,2.2,949,2,102,129,Recomendeitor,235,2,19,share


In [6]:
df.head()

,id_usuario,tipo_usuario,recomendacion_usuario,id_contenido,duracion_contenido,tipo_contenido,valoracion_contenido,numero_views,id_usuario_views,id_contenido_views,minutos_vistos,recomendacion_views,numero_interaccion,id_usuario_interaccion,id_contenido_interaccion,tipo_interaccion
0,214,moderator,Recomendeitor_Plus,153,92,movie,2.5,1,214,153,107,Recomendeitor,1787,214,167,favorite
1,214,moderator,Recomendeitor_Plus,153,92,movie,2.5,1,214,153,107,Recomendeitor,1271,214,180,share
2,214,moderator,Recomendeitor_Plus,153,92,movie,2.5,1,214,153,107,Recomendeitor,1112,214,21,favorite
3,214,moderator,Recomendeitor_Plus,153,92,movie,2.5,1,214,153,107,Recomendeitor,559,214,91,like
4,214,moderator,Recomendeitor_Plus,153,92,movie,2.5,1,214,153,107,Recomendeitor,446,214,80,favorite


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6013 entries, 0 to 6012
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_usuario                6013 non-null   int64 
 1   tipo_usuario              6013 non-null   object
 2   recomendacion_usuario     6013 non-null   object
 3   id_contenido              6013 non-null   int64 
 4   duracion_contenido        6013 non-null   int64 
 5   tipo_contenido            6013 non-null   object
 6   valoracion_contenido      6013 non-null   object
 7   numero_views              6013 non-null   int64 
 8   id_usuario_views          6013 non-null   int64 
 9   id_contenido_views        6013 non-null   int64 
 10  minutos_vistos            6013 non-null   int64 
 11  recomendacion_views       6013 non-null   object
 12  numero_interaccion        6013 non-null   int64 
 13  id_usuario_interaccion    6013 non-null   int64 
 14  id_contenido_interaccion

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id_usuario,6013.0,510.898054,287.277034,1.0,256.0,503.0,767.0,999.0
id_contenido,6013.0,100.245136,57.396154,1.0,52.0,100.0,150.0,200.0
duracion_contenido,6013.0,93.209546,50.445804,10.0,46.0,91.0,139.0,178.0
numero_views,6013.0,1482.385831,859.185107,1.0,735.0,1489.0,2217.0,2998.0
id_usuario_views,6013.0,510.898054,287.277034,1.0,256.0,503.0,767.0,999.0
id_contenido_views,6013.0,100.245136,57.396154,1.0,52.0,100.0,150.0,200.0
minutos_vistos,6013.0,87.421088,51.344747,1.0,43.0,85.0,131.0,179.0
numero_interaccion,6013.0,1010.408282,573.870375,1.0,517.0,1015.0,1514.0,2000.0
id_usuario_interaccion,6013.0,510.898054,287.277034,1.0,256.0,503.0,767.0,999.0
id_contenido_interaccion,6013.0,98.859970,57.819729,1.0,49.0,98.0,148.0,200.0


In [9]:
df.describe(include="O").T

,count,unique,top,freq
tipo_usuario,6013,4,content_creator,1565
recomendacion_usuario,6013,2,Recomendeitor,3075
tipo_contenido,6013,5,short,1362
valoracion_contenido,6013,41,3.9,297
recomendacion_views,6013,2,Recomendeitor_Plus,3083
tipo_interaccion,6013,4,favorite,1603


In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
# df.groupby("id_usuario")[["minutos_vistos","duracion_contenido"]].sum()

In [12]:


#agrupo por id usuario y aplico funciones de agregación al resto porque nos obliga (aunque tenga que pasar a lista( meto set porque así no se pueden repetir valores))
df_usuarios = df.groupby("id_usuario").agg({      #asi puedo aplicar funciones de agregacion a todo junto sin tener que hacerlo por separado
    'tipo_usuario': 'first',  
    'recomendacion_usuario': 'first',  
    'id_contenido': lambda x: list(set(x)),  
    'duracion_contenido': 'sum',  
    'tipo_contenido': lambda x: list(set(x)),  
    'valoracion_contenido': lambda x: list(set(x)),  
    'numero_views': 'count',  
    'minutos_vistos': 'sum',  
    'recomendacion_views':'first',  
    'numero_interaccion': 'count',  
    'tipo_interaccion': lambda x: list(set(x))  
}).reset_index()




In [13]:
df_usuarios.head(5)

,id_usuario,tipo_usuario,recomendacion_usuario,id_contenido,duracion_contenido,tipo_contenido,valoracion_contenido,numero_views,minutos_vistos,recomendacion_views,numero_interaccion,tipo_interaccion
0,1,moderator,Recomendeitor_Plus,"[71, 12, 18, 53, 62]",1304,"[short, live, documentary, movie]","[1.1, 4.6, 1.7, 4.2, 4.7]",20,1380,Recomendeitor_Plus,20,"[comment, favorite, share]"
1,2,moderator,Recomendeitor,[102],23,[documentary],[2.2],1,129,Recomendeitor,1,[share]
2,3,moderator,Recomendeitor_Plus,"[43, 84, 4]",210,"[short, live, series]","[4.1, 4.5, 1.9]",3,188,Recomendeitor_Plus,3,[favorite]
3,4,admin,Recomendeitor_Plus,"[8, 1]",254,"[documentary, live]","[1.0, 4.4]",2,119,Recomendeitor,2,[favorite]
4,5,consumer,Recomendeitor,"[168, 170, 4]",104,"[documentary, series, movie]","[4.9, 3.0, 1.9]",3,199,Recomendeitor,3,[favorite]


Creo una columna con la valoración media

In [14]:
lista=[]
for usuario in range(0, df_usuarios.shape[0]):
    media=round(np.mean(df_usuarios["valoracion_contenido"].values[usuario]),2)
    lista.append(media)
df_usuarios["valoracion_media"]= pd.DataFrame(lista)   

Creo una columna con el numero de tipos de contenido

In [15]:
lista=[]
for usuario in range(0, df_usuarios.shape[0]-1):
    medida=df_usuarios["tipo_contenido"].str.len()
    lista.append(medida)
df_usuarios["numero_tipos_contenido"]= pd.DataFrame(lista[0])   

Creo una columna con si se completó o no la visualización del contenido

In [16]:

df_usuarios["contenido_completado"] = df_usuarios.apply(f.asignar_visualizacion, axis=1)



In [26]:
df_usuarios.sample(3)

,id_usuario,tipo_usuario,recomendacion_usuario,id_contenido,duracion_contenido,tipo_contenido,valoracion_contenido,numero_views,minutos_vistos,recomendacion_views,numero_interaccion,tipo_interaccion,valoracion_media,numero_tipos_contenido,contenido_completado,tasa_no_repeticion
286,356,admin,Recomendeitor_Plus,"[107, 156]",1048,[series],"[2.5, 3.9]",8,904,Recomendeitor,8,"[comment, share]",3.20,1,visualizacion incompleta,0.25
577,713,moderator,Recomendeitor,[166],440,[movie],[4.3],4,376,Recomendeitor_Plus,4,"[like, share]",4.30,1,visualizacion incompleta,0.25
44,58,admin,Recomendeitor,"[160, 33, 164, 69]",978,"[documentary, live, series, movie]","[3.6, 1.0, 3.0, 1.9]",8,934,Recomendeitor,8,[comment],2.38,4,visualizacion incompleta,0.50


Creo una columna con la tasa de NO repetición

In [18]:
lista=[]
for usuario in range(0, df_usuarios.shape[0]-1):
    medida=df_usuarios["id_contenido"].str.len()

    tasa= round(medida/df_usuarios["numero_views"],2)
    lista.append(tasa)
df_usuarios["tasa_no_repeticion"]= pd.DataFrame(lista[0])  

Me he dado cuenta que algunos id_usuario ya faltaban en la base de datos como el 9, 15, 23 etc

In [20]:
df[df["id_usuario"]== 23] 

,id_usuario,tipo_usuario,recomendacion_usuario,id_contenido,duracion_contenido,tipo_contenido,valoracion_contenido,numero_views,id_usuario_views,id_contenido_views,minutos_vistos,recomendacion_views,numero_interaccion,id_usuario_interaccion,id_contenido_interaccion,tipo_interaccion


In [21]:
df_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_usuario              824 non-null    int64  
 1   tipo_usuario            824 non-null    object 
 2   recomendacion_usuario   824 non-null    object 
 3   id_contenido            824 non-null    object 
 4   duracion_contenido      824 non-null    int64  
 5   tipo_contenido          824 non-null    object 
 6   valoracion_contenido    824 non-null    object 
 7   numero_views            824 non-null    int64  
 8   minutos_vistos          824 non-null    int64  
 9   recomendacion_views     824 non-null    object 
 10  numero_interaccion      824 non-null    int64  
 11  tipo_interaccion        824 non-null    object 
 12  valoracion_media        824 non-null    object 
 13  numero_tipos_contenido  824 non-null    int64  
 14  contenido_completado    824 non-null    ob

In [22]:
df_usuarios.describe().T

,count,mean,std,min,25%,50%,75%,max
id_usuario,824.0,506.538835,290.147578,1.00,256.75,504.5,757.25,999.0
duracion_contenido,824.0,680.180825,591.998826,22.00,267.75,513.5,951.25,5873.0
numero_views,824.0,7.297330,5.882006,1.00,3.00,6.0,10.00,49.0
minutos_vistos,824.0,637.940534,555.138386,2.00,250.00,485.0,862.50,4515.0
numero_interaccion,824.0,7.297330,5.882006,1.00,3.00,6.0,10.00,49.0
numero_tipos_contenido,824.0,2.360437,1.039879,1.00,2.00,2.0,3.00,5.0
tasa_no_repeticion,824.0,0.569660,0.299169,0.12,0.33,0.5,1.00,1.0


In [23]:
df_usuarios.describe(include="O").T

,count,unique,top,freq
tipo_usuario,824,4,content_creator,217
recomendacion_usuario,824,2,Recomendeitor,432
id_contenido,824,787,[166],3
tipo_contenido,824,41,"[short, series]",48
valoracion_contenido,824,685,[3.9],10
recomendacion_views,824,2,Recomendeitor_Plus,425
tipo_interaccion,824,15,[favorite],97
valoracion_media,824,199,3.30,22
contenido_completado,824,2,visualizacion incompleta,457


No hay duplicados porque cada fila corresponde a un usuario distinto

In [31]:
df_usuarios["recomendacion_usuario"].value_counts()

recomendacion_usuario
Recomendeitor         432
Recomendeitor_Plus    392
Name: count, dtype: int64

In [30]:
round(df_usuarios["recomendacion_usuario"].value_counts(normalize=True)*100, 2)

recomendacion_usuario
Recomendeitor         52.43
Recomendeitor_Plus    47.57
Name: proportion, dtype: float64

Son bastante parecidos pero para algun test concreto se necesita que ambas tengan el mismo tamaño muestral

Dejo por aquí esto de un lab por si me es útil en algún momento

Como aun rellenando los nulos quedan diferencias en el tamaño de las muestras, re muestreamos aleatoriamente de nuestras muestras para conseguir dos de igual tamaño muestral y ya sacamos el test de Wilcoxon

In [ ]:
# plus = df_usuarios[df_usuarios["recomendacion_usuario"] == "Recomendeitor_Plus"]["Salary"]
# normal = df_usuarios[df_usuarios["recomendacion_usuario"] == "Recomendeitor"]["Salary"]

# min_length = min(len(plus), len(normal))
# plus = plus.sample(n=min_length, random_state=1)
# normal = normal.sample(n=min_length, random_state=1)

# estadistico,p_value = stats.wilcoxon(plus, normal)
# if p_value >= 0.05:
#     print("No hay diferencia")
#     print(".................")
# else:
#     print("Hay diferencia")
#     print(".................")
